Description: Simplified simulator of beam broadening used in Space Physics Lab.

Author: Alexandros Papamatthaiou '21

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [30]:
#Initial beam conditions. The ion source we have in the lab is an IS 40E1

#conversions to SI
eV = 1 /6.242e+18
micro = 1e-6
mm = 1e-3
cm = 1e-2
amu = 1.66054e-27
H_m = 1.00784*amu
He_m = 4.002602*amu

#constants
mu_o = 1.2566370621219e-6
e_o = 8.85418782e-12

#beam energy
en = 3*10**3 *eV  #1keV

#beam current
I = micro #1μA

#beam current density
curr_d = 4*micro*(cm**2)  #4μA/cm^2

#emission current
curr_em = mm  #1mA

#mass of ion
m = 39.948*H_m #Mass of H

#charge
q = eV

#Beam radius
r_b = 3*mm #3mm from CAD (double check)

Equation of motion - Lorentz force: 

$m_{i}\ddot{x} = q_i (E + \dot{x} \times B)$

Gauss's Law:

$\int_{}^{} E \cdot dS \ = \frac{1}{\epsilon_{0}} \int_{}^{} \rho dV \$

Cylindrical coordinates are in order, as the beam is a cylinder. Assuming that the beam is infinitely long and azimuthally symmetric, we conclude that $E_{\theta}, E_{z} = 0$, and we look at the radial field

$2 \pi r_{b} l E_{r} = \frac{\lambda l}{\epsilon_{0}}$

where $\lambda$ is the charge per unit length of the beam, and it is just current over velocity. Hence

$E_{r} = \frac{I_{b}}{2 \pi \epsilon_{0} r_{b} v_{b}}$

The magnetic force can be estimated using Ampere's law.

$\int_{}^{} B \cdot dl \ = \mu_{0} \int_{}^{} J \cdot dS \$

which is $2 \pi r_{b} B_{\theta} = \mu_{0} I_{b}$

In [31]:
#Calculating fundamental quantities

#velocity of beam ions
v_b = np.sqrt(2*en/m)

#Magnetic field
B = mu_o * I / (2*np.pi*r_b)

#Electric field
E = I/(2*np.pi*e_o*r_b*v_b)

#Magnetic force
F_m = q*v_b*B

#Electric force
F_e = q*E

#Ratio of magnetic to electric force is on the order
#of 1e-6, so we can neglect it

F_m/F_e

1.5997425883859174e-07

In [32]:
v_b

119907.33650558705

Since we neglect the magnetic force all we do is plug in the electric force on the equation of motion to get the motion at the edge of the beam:

$m_{i} \ddot{r_{b}} = q_{e} \frac{I_{b}}{2 \pi \epsilon_{0} r_{b}v_{b}}$

It is interesting to note that the force does NOT depend on the type of distribution. As long as it has a set radius and a uniform velocity we are good.

In [33]:
#Integrator to work out the time-evolution of the beam, at distance D from source
#D is the distance traversed by the beam.
#r0 is the initial radius of the beam. 
#port is the intensity of the beam that we allow. 

#This is important for getting the beam broadening after the knife edge,
#which may allow only a portion of the beam through.

def Beam(r0,D,port):
    
    #Simple RK4 integrator. Could have used a package but this is more explicit
    
    global e_o,q,I,r_b,v_b,m
    
    dt = 0.00000001
    
    t_final = D/v_b  #the beam won't get large enough that this approximation becomes wrong
    N = int(round((t_final)/dt)) 
    t = np.linspace(0, t_final, N+1)
    
    r = np.zeros((N+1, 1))
    v = np.zeros((N+1, 1))
    
    r[0, :] = r0  # initialize beam size
    v[0, :] = 0  # initialize velocity
    
    def func(tau, x, xdot):
        return q*port*I/(2*np.pi*x*e_o*v_b*m)
    
    for i in range(0, len(t)-1):
        k1x = v[i]
        k1v = func(t[i],r[i],v[i])
        k2x = v[i]+dt*k1v/2
        k2v = func(t[i] +dt/2,r[i]+dt*k1x/2,v[i]+dt*k1v/2)
        k3x = v[i]+dt*k2v/2
        k3v = func(t[i] +dt/2,r[i]+dt*k2x/2,v[i]+dt*k2v/2)
        k4x = v[i]+dt*k3v
        k4v = func(t[i]+dt,r[i]+dt*k3x,v[i]+dt*k3v)
        r[i+1] = r[i] + dt/6 * (k1x+2*k2x+2*k3x+k4x)
        v[i+1] = v[i] + dt/6 * (k1v+2*k2v+2*k3v+k4v)
        
    return r

In [34]:
#example beam broadening after 5m
Beam(r_b,5,1)

array([[0.003     ],
       [0.00300001],
       [0.00300002],
       ...,
       [0.04577898],
       [0.04579297],
       [0.04580697]])

In [19]:
#Knife edge that cuts a portion of the beam out. r is the final radius before the knive-edge
#port is the portion of the distribution (in σ) the knife edge cuts out. The function
#spits out the ideal radius of a knife-edge hole in mm.

def Knife_Edge(r,port):
    
    #we assume that the beam profile is Gaussian and a particle at the rim is at 3σ
    #this is generally a good approximation as the ion source produces a Gaussian beam
    
    sigma = r[-1]/3 #The r is at 3σ so σ is conveniently r/3. remember μ = 0
    
    if port > 3: 
        r_new = 3*sigma #limit in case user asks for more than 3σ as we have set the boundary ion to be at 3σ
    else:
        r_new = port*sigma #new r is limited to a portion of the previous gaussian, like σ/2
    
    print("Knife edge hole should be about: " + str(np.round(r_new[0],4)*1e+3) + " mm in radius")
    return r_new

All parts are completed. Now a test run.

The beam calculation has three parts:

1. A first flight path 

2. Knife edge passage

3. A second flight path

Output is given in mm

In [11]:
#Beam path: Initial beam passes through knife edge at 2 meters, then continues for another 3 meters.
#r_f is the final radius as it hits SWAPI. Knife edge allows through 3σ

r_1 = r_b
r_2 = Beam(r_1,2,1)
r_3 = Knife_Edge(r_2,3)
r_f = Beam(r_3,3,1)
r_f[-1]*1e+3 #beam radius in mm

Knife edge hole should be about: 12.6 mm in radius


array([19.38737473])

In [12]:
#Second test run. Same as before allows 1 σ. Notice the beam at the second flight path is attenuated to 68%, because
#the knife edge allows only 1σ through.

r_1 = r_b
r_2 = Beam(r_1,2,1)
r_3 = Knife_Edge(r_2,1)
r_f = Beam(r_3,3,0.682)
r_f[-1]*1e+3 #beam radius in mm

Knife edge hole should be about: 4.2 mm in radius


array([15.27006571])

The beam gets only slightly narrower if we minimize it at the knife-edge, as the intensity of the beam is lower, but we deal again with a tighter distribution than before so it gets spread out faster. So we have a beam that is only 21% smaller but we only get 68% in intensity!

TL;DR: It's worth it to keep all the beam instead of cutting it at the knife-edge